In [1]:
import re
from bs4 import BeautifulSoup
import requests
import html5lib
import json

In [2]:
url = 'https://www.cna.com.tw/list/aspt.aspx'
res = requests.get(url)
soup = BeautifulSoup(res.text,'html5lib')
n=0
labels=[]
while n < len(soup.select_one('#pnProductNavContents > ul').findAll('li')):
    try :
        lab = soup.select_one('#pnProductNavContents > ul').findAll('li')[n].find('a',{'class':'first-level'}).text
        url =soup.select_one('#pnProductNavContents > ul').findAll('li')[n].find('a',{'class':'first-level'})['href']
    except:(TypeError,IndentationError)
    labels.append((url,lab))
    n += 1
labels = list(set(labels))
labels.remove(('http://cnavideo.cna.com.tw', '影音'))
labels.remove(('/list/aall.aspx', '即時'))
labels.append(('https://www.cna.com.tw/list/aall.aspx', '即時'))
#labels.remove(('https://www.cna.com.tw/keyword/九合一.aspx', '九合一'))
labels.remove(('https://www.cna.com.tw/list/sp.aspx', '專題'))
labels.remove(('https://www.cna.com.tw/list/aall.aspx', '即時'))

labs = []
for item in labels:
    item = [item[0],item[1]]

    item[0] = re.search(r'list\/\w+',item[0]).group()[5:]
    labs.append(item)

In [3]:
def payload(item):
    payload={
        'id': '1056893641102242',
    'ev': 'Microdata',
    'dl': item[0],
    'rl': 'https://www.cna.com.tw/',
    'if': 'false',
    'ts': '1541087113735',
    'cd[Schema.org]': [],
    #'cd[OpenGraph]': {"og:url":"https://www.cna.com.tw/list/ahel.aspx","og:title":"生活 | 中央社 CNA","og:description":"想知道統一發票、大樂透、威力彩開獎號碼、各種醫療保健、交通氣象、颱風動態等生活資訊，及升學考試、教育政策、課綱改革等文教訊息，通通都在中央社生活新聞。","og:image:height":"200","og:image:width":"200","og:image":"https://img5.cna.com.tw/www/images/pic_fb.jpg","og:type":"article","article:author":"https://www.facebook.com/cnanewstaiwan","article:publisher":"https://www.facebook.com/cnanewstaiwan"}
    #'cd[Meta]': {"title":"\n\t生活 | 中央社 CNA\n","meta:description":"想知道統一發票、大樂透、威力彩開獎號碼、各種醫療保健、交通氣象、颱風動態等生活資訊，及升學考試、教育政策、課綱改革等文教訊息，通通都在中央社生活新聞。","meta:keywords":"News, 新聞, 即時新聞, 中央社"}
    'cd[DataLayer]': [],
    'cd[JSON-LD]': [],
    'sw': '1280',
    'sh': '5000',#'720',
    'v': '2.8.30',
    'r': 'stable',
    'a': 'tmgoogletagmanager',
    'ec': '1',
    'o': '30',
    'fbp': 'fb.2.1541074934894.620990608',
    'it': '1541087112886',
    'coo': 'false',
    'es': 'automatic',
    }
    return payload

In [4]:
labs_href=[]
for item in labs:
    soups=[]
    n=1
    while n < 13:
        url = 'https://www.cna.com.tw/cna2018api/api/simplelist/categorycode/'+item[0]+'/pageidx/'+str(n)
        res = requests.get(url)
        soup = BeautifulSoup(res.text,'html5lib')
        body = soup.findAll('body')[0].getText()
        PageUrl = re.findall(r'\"PageUrl\"\:\S[^\,]+',body)
        urls = [txt[11:-1] for txt in PageUrl]
        soups.extend(urls)
        n += 1
        if len(urls) < 20:
            break
    labs_href.append([soups,item[1]])

In [5]:
news={}
newslist=[]
for lab in labs_href:
    lab_name = lab[1]
    lab_urls = lab[0]
    n = 0
    for url in lab_urls:
        res = requests.get(url)
        soup = BeautifulSoup(res.text,'html5lib')
        txt = ""
        try:
            title = soup.find_all('div',{'class':'centralContent'})[0].find_all('h1')[0].getText()
            for s in soup.find_all('div',{'class':'paragraph'})[0].findAll('p'):
                txt=txt+s.getText()
        except:IndexError
        research = re.search(r'[\）\)\：\:\》](.+[\。\!\?\s])+',txt)
        if research == None or len(research.group())<2:
            print(txt)
        else:
            txt = re.search(r'[\）\)\：\:\》](.+[\。\!\?\s])+',txt).group()[1:]

        page = [[title,txt],lab_name]
        newslist.append(page)
        news[(title,txt)]=lab_name
        n += 1
        if n % 20 == 0:
            print((lab_name,n),end=",")

('文化', 20),('文化', 40),('文化', 60),('文化', 80),('文化', 100),('科技', 20),('科技', 40),('科技', 60),('科技', 80),('科技', 100),('政治', 20),('政治', 40),('政治', 60),('政治', 80),('政治', 100),('國際', 20),('國際', 40),('國際', 60),('國際', 80),('國際', 100),('產經', 20),('產經', 40),('產經', 60),('產經', 80),('產經', 100),('社會', 20),('社會', 40),('社會', 60),('社會', 80),('社會', 100),('證券', 20),('證券', 40),('證券', 60),('證券', 80),('證券', 100),('娛樂', 20),('娛樂', 40),('娛樂', 60),('娛樂', 80),('娛樂', 100),('兩岸', 20),中央社駐上海特派員陳家倫/1月10日現年92歲的上海海關大鐘近來開始誤點，去年8月還一度中暑暈厥了好幾小時。衰老的大鐘預計於今年接受英國原廠「健檢」，不論結果是搶救或無痛地走向終點，都將轟動上海灘。上海海關大樓是外灘知名歷史建築，頂部的大鐘與倫敦大笨鐘是「姊妹」，由英國同一家公司生產。據稱，上海海關大鐘當年總造價高達5000多兩白銀，於1927年8月從倫敦運到上海，原包裝木箱連同大鐘重達6.25噸。1928年元旦凌晨1時，海關大鐘敲響了第一聲，從此以後，鐘聲就日以繼夜地陪伴著上海人。對於早年生活並不富裕的中國民眾而言，別說手上戴個表，連在家掛個時鐘都是奢侈，噹噹響的鐘聲因而成了市民珍視的「公共財」。「以前再遠都聽得到呦」，62歲的上海徐姓民眾從小在靜安寺附近長大，住家距離外灘約5公里遠。他回憶，學齡前就對海關大鐘的鐘聲有印象，但受到都市化的高樓阻絕、汽車雜音干擾，老家現已聽不到鐘聲。至於原先住在外灘一帶的居民，因都市計畫與拆遷的關係，也從海關大鐘的「搖滾區」撤出，接替而上的是大批境外觀光客，連帶將海關大鐘推向世界，鐘聲敲進了更多人的耳裡，並迴盪在老上海的記憶中。享受世人矚目的同時，邁入92歲的海關大鐘也面臨著歲月的考驗，自去年起，就開始不太準時，常有熱心民眾去電通報

In [6]:
import jieba
import nltk

In [7]:
stopwords=[]
with open('../30/txt/stopwords_ch.txt', 'r', encoding='UTF-8') as file:
    for data in file.readlines():
        data = data.strip()
        stopwords.append(data)

In [8]:
lab_tags ={}
prelab = ''
books = list(news.items())
for book in books:
    for lab in ['生活','運動','政治','科技','國際','兩岸','產經','社會','地方','娛樂','證券','文化']:
        if book[1]==lab:
            if prelab != lab:
                txt=''
            txt = txt+book[0][1]
            lab_tags[book[1]] = txt
            prelab = lab

In [86]:
filterwords=[
    '年','月','中央社','記者','編輯','譯者','日電'
    ,'\n',' ','「','」','『','』'
    ,'今天','說','表示','年','月','日','後','前','時','應','中','今年'
    ,'兩','下','人','最','次','再','先','盡量'
]

In [10]:
articles = []
for news in newslist:
    articles.append((news[0][1],news[1]))
    

In [13]:
books = []


for n in range(len(articles)):

    txt_dic = {}
    txt = articles[n]
    txt_dic[txt[0]] = txt[1]
    books.append(txt_dic)


In [57]:
def get_tags(books=books):
    tags = {}  
    labs = list(set([v for b in books for v in b.values()]))
    for lab in labs:
        txt = ""
        for book in books:
            if list(book.values())[0] == lab:
                txt += list(book.keys())[0]
        tags[lab] = txt
    return tags

In [60]:
txt_tags = get_tags(books)

In [111]:
def get_fwords(txt="",top=300,twotop = 20,output_values='items'):
    seg_list = jieba.cut(txt, cut_all=False)
    txt = ",".join(seg_list)
    tokens = txt.split(',')

    tokens = [re.match(r'\w+',w).group() for w in tokens if (w not in stopwords and w not in filterwords) and re.match(r'\w+',w) != None]
#    tokens = list(filter(lambda a: a not in stopwords and a not in filterwords and a.isalpha() == True,tokens))#and len(a)>1, tokens))
    one = nltk.FreqDist(tokens).most_common(top)

    lab_alltwowords = list(nltk.bigrams(tokens))
    two = [(str(i[0][0])+str(i[0][1]),i[1]) for i in nltk.FreqDist(lab_alltwowords).most_common(twotop)]

    wordcount_dict = {}
    for com in [two,one]:
        for i in com:
            if re.search(r'\D+',i[0]) != None:
                wordcount_dict[i[0]] = i[1]
    if output_values == 'items':
        return list(wordcount_dict.items())
    elif output_values == 'dic':
        return wordcount_dict

In [125]:
def get_fwords_tags(output_values='items'):
    
    fwords_tags = {}
    for i in list(txt_tags.items()):
        fwords_tags[i[0]]=get_fwords(i[1],top=300,output_values=output_values) 
    return fwords_tags

In [126]:
fwords_tags_items = get_fwords_tags(output_values='items')

In [127]:
fwords_tags_dic = get_fwords_tags(output_values='dic')

In [190]:
fwords_tags_dic['兩岸']

{'40週年': 16,
 'iPhone': 16,
 '一國兩制': 94,
 '一國兩制台灣': 13,
 '上海': 54,
 '上述': 17,
 '下降': 17,
 '中共': 59,
 '中國': 397,
 '中國人': 17,
 '中國國家': 26,
 '中國大陸': 54,
 '中央': 42,
 '中小學': 19,
 '中小學教師': 14,
 '中心': 36,
 '中美': 25,
 '主席': 27,
 '主席習近平': 13,
 '主張': 20,
 '主教': 15,
 '主權': 21,
 '主義': 18,
 '主要': 26,
 '九二': 68,
 '九二共識': 68,
 '事': 14,
 '事件': 16,
 '些': 28,
 '交流': 16,
 '人口': 27,
 '人員': 48,
 '人士': 24,
 '人民幣': 29,
 '代表': 25,
 '令': 17,
 '以上': 15,
 '企業': 31,
 '伊斯蘭教': 18,
 '位': 20,
 '低': 16,
 '作出': 19,
 '作戰': 15,
 '做': 16,
 '價格': 16,
 '億': 26,
 '億元': 20,
 '元': 79,
 '兆': 16,
 '內容': 33,
 '全國': 23,
 '全球': 17,
 '兩岸': 72,
 '公共': 16,
 '公司': 44,
 '公布': 25,
 '公開': 16,
 '共同': 24,
 '共識': 90,
 '出現': 16,
 '分析': 22,
 '利益': 14,
 '制度': 27,
 '副': 18,
 '副中心': 17,
 '加強': 17,
 '包括': 42,
 '北京': 144,
 '北京市': 19,
 '北京方面': 34,
 '協商': 14,
 '協會': 17,
 '占': 17,
 '印象': 15,
 '卻': 17,
 '原則': 20,
 '去年': 65,
 '參加': 22,
 '反對': 14,
 '召開': 20,
 '台': 50,
 '台灣': 239,
 '台灣同胞書': 16,
 '同意': 17,
 '同胞書': 16,
 '名': 54,
 '告': 16,
 '告台灣': 15,
 '呼籲':

In [191]:
fwords_tags_items['兩岸']

[('九二共識', 68),
 ('中國大陸', 54),
 ('蔡總統', 37),
 ('北京方面', 34),
 ('中國國家', 26),
 ('億元', 20),
 ('經濟成長', 18),
 ('副中心', 17),
 ('台灣同胞書', 16),
 ('40週年', 16),
 ('告台灣', 15),
 ('行政長官', 14),
 ('中小學教師', 14),
 ('教師資格', 14),
 ('國家主席', 13),
 ('主席習近平', 13),
 ('國家統一', 13),
 ('一國兩制台灣', 13),
 ('總統蔡英文', 13),
 ('中國', 397),
 ('台灣', 239),
 ('香港', 202),
 ('北京', 144),
 ('報導', 125),
 ('國家', 123),
 ('大陸', 107),
 ('一國兩制', 94),
 ('共識', 90),
 ('指出', 87),
 ('習近平', 84),
 ('元', 79),
 ('已', 77),
 ('政府', 76),
 ('兩岸', 72),
 ('九二', 68),
 ('經濟', 68),
 ('去年', 65),
 ('民主', 62),
 ('提出', 60),
 ('美國', 59),
 ('中共', 59),
 ('總統', 57),
 ('認為', 56),
 ('工作', 55),
 ('名', 54),
 ('上海', 54),
 ('新', 53),
 ('接受', 52),
 ('方面', 51),
 ('進行', 51),
 ('台', 50),
 ('問題', 50),
 ('關係', 49),
 ('人員', 48),
 ('大學', 48),
 ('發展', 47),
 ('城市', 47),
 ('會議', 46),
 ('統一', 45),
 ('發表', 45),
 ('市場', 45),
 ('相關', 45),
 ('公司', 44),
 ('包括', 42),
 ('中央', 42),
 ('國歌', 42),
 ('政策', 41),
 ('委員會', 41),
 ('項', 39),
 ('社會', 39),
 ('民眾', 39),
 ('政治', 38),
 ('未', 38),
 ('蔡', 3

In [187]:
def remove_mix(fword_value="items",lab_fwords={}):

    dic3={}
    for lab in lab_fwords.keys():   
        word_content2 = lab_fwords[lab].copy()
        if fword_value == "dic":
            word_content2 = list(word_content2.items())
        for lab2 in lab_fwords.keys():    
            if lab != lab2:
                if fword_value == "items":
                    word_content = [w for w in lab_fwords[lab] if (w[0] in [a[0] for a in lab_fwords[lab2]])]
                    for w in word_content:
                        try:
                            word_content2.remove(w)                  
                        except ValueError:
                            pass
                        
                        
            if fword_value == "items":
                dic3[lab] = word_content2 
            if fword_value == "dic":
                ddd = {}
                for i in word_content2:
                    ddd[i[0]]=i[1]
                dic3[lab] = ddd

    return dic3

In [188]:
fword_dic = remove_mix(lab_fwords=fwords_tags_dic,fword_value='dic')

In [189]:
fword_items = remove_mix(lab_fwords=fwords_tags_items,fword_value='items')

In [192]:
import random
random.shuffle(books)

In [253]:
def pos_fwords(article,top=200):

    features={}
    fwords = get_fwords(article,top)
    for w in fwords :
        w = w[0]
        if w in [i[0] for i in [t for v in fword_items.values() for t in v]]:
            features[w] = True
    return features

In [255]:
def get_features(dic={},top=200):

    article = list(dic.keys())[0]
    labname = list(dic.values())[0]
    features = pos_fwords(article,top)

    if len(list(features.keys())) < 2:
        feature_item = [{},labname]
    else:
        feature_item = [features,labname]

    return feature_item

In [256]:
def get_featuresets(books=books,top=200):
    txt_featuresets = []
    for book in books:

        txt_featureset = get_features(book,top=top)
        txt_featuresets.append(txt_featureset)
    return  txt_featuresets

In [257]:
txt_featuresets = get_featuresets()

In [258]:
txt_featuresets

[[{'之前': True,
   '出席': True,
   '創下': True,
   '原則': True,
   '年輕': True,
   '成績': True,
   '業餘': True,
   '母親': True,
   '父親': True,
   '粉絲': True,
   '職業': True,
   '賽事': True},
  '運動'],
 [{'主任': True,
   '之前': True,
   '二選區': True,
   '卓榮泰': True,
   '印象': True,
   '參選人': True,
   '合唱': True,
   '唱': True,
   '士林': True,
   '大同': True,
   '對手': True,
   '志偉': True,
   '改編': True,
   '最終': True,
   '柯文哲': True,
   '歌手': True,
   '歌曲': True,
   '炳甫': True,
   '版本': True,
   '瑜': True,
   '男': True,
   '秒': True,
   '笑說': True,
   '藉': True,
   '補選參選人': True,
   '週': True,
   '陳思宇': True,
   '順利': True},
  '地方'],
 [{'公噸': True,
   '台電': True,
   '品質': True,
   '大聯盟': True,
   '局長': True,
   '操作': True,
   '機制': True,
   '瑜': True,
   '科': True,
   '空氣品質': True,
   '策略': True,
   '興達': True,
   '興達電廠': True,
   '落實': True,
   '附近': True,
   '電廠': True},
  '地方'],
 [{'上漲': True,
   '下跌': True,
   '之上': True,
   '個股': True,
   '光學': True,
   '加權': True,
   '加碼': True,
   '受惠': True,
   '台

In [259]:
txt_featuresets = txt_featuresets.copy()
size = int(len(txt_featuresets)*0.7)
train_set, test_set = txt_featuresets[:size], txt_featuresets[size:]
txt_classifier = nltk.NaiveBayesClassifier.train(train_set)
nltk.classify.accuracy(txt_classifier, test_set)

0.7222222222222222

In [275]:
[i[0]  for v in fwords_tags_items.values() for i in v]

['億元',
 '30元',
 '去年12',
 '億美元',
 '經常性薪資',
 '兌美元',
 '歷史新高',
 '新台幣兌',
 '合併營收',
 '部分',
 '元年增',
 '去年季',
 '國營事業',
 '1元',
 'MyMusic',
 '自由貿易',
 'TaiwanPlus',
 '吳清典',
 '董事會決議',
 '智慧型手機',
 '元',
 '億',
 '去年',
 '公司',
 '台灣',
 '市場',
 '中國',
 '經濟',
 '美國',
 '全球',
 '新台幣',
 '指出',
 '新',
 '已',
 '美元',
 '貿易',
 '投資',
 '成長',
 '政府',
 '分',
 '調查',
 '營收',
 '企業',
 '產業',
 '影響',
 '高',
 '目前',
 '華為',
 '萬',
 '發展',
 '約',
 '科技',
 '手機',
 '達',
 '集團',
 '服務',
 '雙方',
 '問題',
 '事業',
 '電動車',
 '國家',
 '國際',
 '遊戲',
 '薪資',
 '包括',
 '產品',
 '合作',
 '超過',
 '報導',
 '工作',
 '磋商',
 '5G',
 '提供',
 '季',
 '貝佐斯',
 '網路',
 '主要',
 '政策',
 '台電',
 '獎金',
 '進行',
 '汽車',
 '銀行',
 '台商',
 '業者',
 '透過',
 '虧損',
 '協助',
 '董事會',
 '陸資',
 '新高',
 '旗下',
 '中心',
 '亞馬遜',
 '印度',
 '蘋果',
 '活動',
 '未來',
 '重要',
 '項',
 '宣布',
 '日本',
 '增加',
 '經濟部',
 '年增',
 'iPhone',
 '表現',
 '共同',
 '英國',
 '萬元',
 '工程',
 '希望',
 '認為',
 '是否',
 '外匯',
 '能源',
 '達成',
 '合併',
 '歷史',
 '銷售',
 '領域',
 '總經理',
 '計畫',
 '家',
 '億美',
 '應用',
 '價格',
 '名',
 '需求',
 '持續',
 '平台',
 '民眾',
 '景氣',
 '機會',
 '董事長',
 '相當',
 '智慧',
 '

In [270]:
fwords_tags_items

{'兩岸': [('九二共識', 68),
  ('中國大陸', 54),
  ('蔡總統', 37),
  ('北京方面', 34),
  ('中國國家', 26),
  ('億元', 20),
  ('經濟成長', 18),
  ('副中心', 17),
  ('台灣同胞書', 16),
  ('40週年', 16),
  ('告台灣', 15),
  ('行政長官', 14),
  ('中小學教師', 14),
  ('教師資格', 14),
  ('國家主席', 13),
  ('主席習近平', 13),
  ('國家統一', 13),
  ('一國兩制台灣', 13),
  ('總統蔡英文', 13),
  ('中國', 397),
  ('台灣', 239),
  ('香港', 202),
  ('北京', 144),
  ('報導', 125),
  ('國家', 123),
  ('大陸', 107),
  ('一國兩制', 94),
  ('共識', 90),
  ('指出', 87),
  ('習近平', 84),
  ('元', 79),
  ('已', 77),
  ('政府', 76),
  ('兩岸', 72),
  ('九二', 68),
  ('經濟', 68),
  ('去年', 65),
  ('民主', 62),
  ('提出', 60),
  ('美國', 59),
  ('中共', 59),
  ('總統', 57),
  ('認為', 56),
  ('工作', 55),
  ('名', 54),
  ('上海', 54),
  ('新', 53),
  ('接受', 52),
  ('方面', 51),
  ('進行', 51),
  ('台', 50),
  ('問題', 50),
  ('關係', 49),
  ('人員', 48),
  ('大學', 48),
  ('發展', 47),
  ('城市', 47),
  ('會議', 46),
  ('統一', 45),
  ('發表', 45),
  ('市場', 45),
  ('相關', 45),
  ('公司', 44),
  ('包括', 42),
  ('中央', 42),
  ('國歌', 42),
  ('政策', 41),
  ('委員會', 41

In [294]:
from collections import Counter
def get_top(article,top=5,get=20):
    fwords = get_fwords(article)
    txt_comanwer = txt_classifier.classify(pos_fwords(article))
#    [v[0] d.values for d in fwords_tags_items for v in d.values()]
    labcount = []
    for i in fwords:
        w = i[0]
        for lab in fwords_tags_items.keys():
            if w in [wi[0] for wi in fwords_tags_items[lab]]:
                labcount.append(lab)
#        print(w,labcount)
    print(Counter(labcount).most_common(5),txt_comanwer)
    return Counter(labcount).most_common(5)

In [295]:
get_top(testtxt)

[('運動', 18), ('娛樂', 17), ('國際', 17), ('文化', 16), ('產經', 13)] 運動


[('運動', 18), ('娛樂', 17), ('國際', 17), ('文化', 16), ('產經', 13)]

In [267]:
txt_top = [list(get_top(txt,175,175).values())[0] for txt in [list(book.keys())[0] for book in books]]

NameError: name 'txt_fwords' is not defined

In [264]:
[list(book.keys())[0] for book in books]

['被日媒譽為圍棋界天才少女的日本業餘棋士仲邑堇目前年僅9歲，已獲准在4月1日滿10歲時成為日本棋院職業棋士，將創下日本棋院最年輕職業棋士紀錄。日本產經新聞報導，日本棋院今天宣布，出身大阪府、目前在大阪市就讀小學4年級的仲邑堇，將在4月1日成為職業棋士。仲邑堇到4月1日將滿10歲，日本棋院之前最年輕職業棋士紀錄，是現年20歲的藤澤里菜在11歲半時所創。出席這場記者會的仲邑堇說，接下來要請大家多多指教。日本棋院理事長團宏明談到允許仲邑堇成為職業棋士時說，期待仲邑堇能在世界舞台上大放異彩，也希望能吸引更多的圍棋粉絲。仲邑堇的父親是日本棋院關西總本部的仲邑信也（九段），現年45歲；母親是前圍棋指導員仲邑幸，現年38歲。仲邑堇3歲時跟媽媽學習圍棋規則，5歲就在業餘賽事創下最年輕的獲勝紀錄，很早就受到各界關注。要成為圍棋職業棋士，原則上必須是日本棋院或關西棋院的院生，並跟多位高手對弈獲勝取得一定成績。仲邑堇曾是關西棋院院生，日本棋院這次認為仲邑堇到目前為止的成績，已與職業棋士相當，允許仲邑堇從4月起進入職業世界。',
 '無黨籍台北市立委補選參選人陳思宇公布競選歌曲「讓白色發光」，並邀歌手白芯羽、「學姐」黃瀞瑩及台北市長柯文哲合唱，期待藉此加深選民印象並轉化為支持度。陳思宇今天舉行競選歌曲「讓白色發光」發布記者會，並在日前搶先釋出24秒無聲版本錄音花絮 ，歌詞以白色力量、推倒有顏色高牆為主軸，並融入柯文哲常說的do the right thing，do things right，歌詞最後還有「嗡嗡」，柯市府團隊特色展露無遺。詞曲創作者找來老天鵝改編「憨智回頭」的男主唱「郭子」，並由郭子擔任製作人，陳思宇競選團隊新聞部主任蔡峻維說，郭子創作靈感從陳思宇的競選宣言而來，從譜曲、創作到錄製，約花1週完成 。今天陳思宇與白芯羽、黃瀞瑩及柯文哲合唱「讓白色發光」，柯文哲上台前頻頻對陳思宇說「我沒有卡拉OK就不會唱」、「等等都要靠妳了」，最終4人順利合體。陳思宇表示，在錄音之前其實對自己的歌喉非常沒信心，但進錄音間後發現沒有被挑剔音準，並笑說可能是柯文哲給她信心，「畢竟阿北都能唱了」。陳思宇近來因為掃街緣故聲音沙啞，但她表示有在試著用丹田發力並放慢說話速度，狀況稍好一些。台北市第二選區（士林、大同）立委補選將在27日舉行，陳思宇的競爭對手有民進黨何志偉與國民黨陳炳甫，前者有民進黨主席卓榮